# ClipMap Visualization: DevOps/SRE

**Purpose:** This notebook visualizes the aggregated ClipCard data to identify systemic patterns and recurring failure modes for the DevOps/SRE silo.

It focuses on answering questions like:
- What are the most common types of hazards we face?
- Is there a relationship between certain hazards and their outcomes?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the aggregated ClipCard data
try:
    cards_df = pd.read_csv('../../data/clipmap/cards.csv')
    events_df = pd.read_csv('../../data/clipmap/events.csv')
    # Filter for devops_sre domain if other domains exist
    sre_cards = cards_df[cards_df['domain'] == 'dfm-devops-sre'].copy()
    # Merge the two dataframes to link hazards with outcomes
    merged_df = pd.merge(sre_cards, events_df, left_on='id', right_on='card_id')
except FileNotFoundError as e:
    print(f'ERROR: {e.filename} not found. Please ensure the data exists.')
    merged_df = pd.DataFrame() # Empty dataframe to prevent further errors

### Hazard Frequency

First, let's analyze the types of hazards we are logging. For simplicity, we'll extract keywords from the `telos` (hazard description) field.

In [ ]:
if not merged_df.empty:
    # Extract keywords from the hazard description
    # This is a simple approach; a more advanced method would use NLP
    merged_df['hazard_keywords'] = merged_df['telos'].str.findall(r'(billing|database|API|downstream|corruption|charges)')
    merged_df['hazard_keywords'] = merged_df['hazard_keywords'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else 'other')

    plt.figure(figsize=(10, 6))
    sns.countplot(y='hazard_keywords', data=merged_df, order=merged_df['hazard_keywords'].value_counts().index)
    plt.title('Frequency of Hazard Types in DevOps/SRE')
    plt.xlabel('Count')
    plt.ylabel('Hazard Keyword')
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.show()
else:
    print('Data is empty. Cannot plot hazard frequency.')

### Hazard-Outcome Heatmap

This heatmap shows the relationship between hazard types and their final outcomes. It can help us spot if certain hazards are more likely to result in incidents being prevented (a good thing) or other outcomes.

In [ ]:
if not merged_df.empty:
    # Create a contingency table (crosstab) of hazards and outcomes
    contingency_table = pd.crosstab(merged_df['hazard_keywords'], merged_df['outcome'])

    plt.figure(figsize=(12, 7))
    sns.heatmap(contingency_table, annot=True, cmap='YlGnBu', fmt='d')
    plt.title('Heatmap of Hazard Keywords vs. Outcomes')
    plt.xlabel('Outcome')
    plt.ylabel('Hazard Keyword')
    plt.show()
else:
    print('Data is empty. Cannot plot heatmap.')